# Train Networks

- Train SoftMax or Multi-BCE classifier for the EEG diagnosis classification
    - CAUEEG-task1 benchmark: Classification of **Normal**, **MCI**, and **Dementia** symptoms
    - CAUEEG-task2 benchmark: Classification of **Normal** and **Abnormal** symptoms
- `Weights and Biases` sweep is used for hyperparameter search

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# Load some packages
import os
import json
from copy import deepcopy
import gc
import time

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pprint
import wandb

# custom package
from datasets.caueeg_dataset import *
from datasets.caueeg_script import *
from models import *
from train import *

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): 
    print('cuda is available:', torch.cuda.get_device_name(0))
else: 
    print('cuda is unavailable.') 

PyTorch version: 1.11.0+cu113
cuda is available: NVIDIA GeForce RTX 3090


-----

## Set the default configuration for building datatset

In [4]:
cfg_data = {}
cfg_data['device'] = device
cfg_data['task'] = 'task1'
cfg_data['dataset_path'] = r'local/dataset/02_Curated_Data_220419/'
cfg_data['file_format'] = 'memmap'  # 'feather', 'memmap'
cfg_data['load_event'] = False
cfg_data['latency'] = 200 * 10      # 10 seconds
cfg_data['seq_length'] = 200 * 10  # 10 seconds
cfg_data['longer_seq_length'] = 200 * 10 * 6 # 60 seconds
cfg_data['crop_multiple'] = 4
cfg_data['EKG'] = 'O'
cfg_data['photic'] = 'X'
cfg_data['input_norm'] = 'dataset'  # 'datatset', 'datapoint', 'no'
cfg_data['awgn'] = 5e-2
cfg_data['mgn'] = 1e-4
cfg_data['awgn_age'] = 5e-2

if '3090' in torch.cuda.get_device_name(0):
    cfg_data['minibatch'] = 256
elif '2080' in torch.cuda.get_device_name(0):
    cfg_data['minibatch'] = 128
elif '1070' in torch.cuda.get_device_name(0):
    cfg_data['minibatch'] = 96

In [5]:
cfg_data_temp = deepcopy(cfg_data)
_ = build_dataset_for_train(cfg_data_temp, verbose=True)

in_channels = cfg_data_temp['preprocess_train'](next(iter(_[0])))['signal'].shape[1]
out_dims = len(cfg_data_temp['class_label_to_name'])

del _

transform: Compose(
    EegRandomCrop(crop_length=2000, length_limit=10000000, multiple=4, latency=2000, return_timing=False)
    EegDropChannels(drop_index=[20])
    EegToTensor()
)

----------------------------------------------------------------------------------------------------

transform_longer: Compose(
    EegRandomCrop(crop_length=12000, length_limit=10000000, multiple=4, latency=2000, return_timing=False)
    EegDropChannels(drop_index=[20])
    EegToTensor()
)

----------------------------------------------------------------------------------------------------


task config:
{'class_label_to_name': ['Normal', 'MCI', 'Dementia'],
 'class_name_to_label': {'Dementia': 2, 'MCI': 1, 'Normal': 0},
 'task_description': 'Classification of [Normal], [MCI], and [Dementia] '
                     'symptoms.',
 'task_name': 'CAUEEG-task1 benchmark'}

 ---------------------------------------------------------------------------------------------------- 

train_dataset[0].keys():
dict_keys

In [6]:
pprint.pprint(cfg_data_temp, width=250)

{'EKG': 'O',
 'age_mean': tensor([71.1663]),
 'age_std': tensor([9.7996]),
 'awgn': 0.05,
 'awgn_age': 0.05,
 'class_label_to_name': ['Normal', 'MCI', 'Dementia'],
 'class_name_to_label': {'Dementia': 2, 'MCI': 1, 'Normal': 0},
 'crop_multiple': 4,
 'dataset_name': 'CAUEEG dataset',
 'dataset_path': 'local/dataset/02_Curated_Data_220419/',
 'device': device(type='cuda'),
 'file_format': 'memmap',
 'input_norm': 'dataset',
 'latency': 2000,
 'load_event': False,
 'longer_seq_length': 12000,
 'mgn': 0.0001,
 'minibatch': 256,
 'photic': 'X',
 'preprocess_test': Sequential(
  (0): EegToDevice(device=cuda)
  (1): EegNormalizeAge(mean=tensor([71.1663]),std=tensor([9.7996]),eps=1e-08)
  (2): EegNormalizeMeanStd(mean=tensor([ 0.0506,  0.0013, -0.0268,  0.0043,  0.0031, -0.0424,  0.0146,  0.0255,
           0.0181,  0.0191,  0.0170, -0.0620, -0.0380, -0.0137,  0.1280, -0.0564,
           0.0593, -0.0222,  0.0019, -0.0239]),std=tensor([45.2187, 20.4865, 11.5672, 11.4781, 15.5555, 47.7741, 19.82

-----

## Define Network Models

In [7]:
cfg_common_model = {'in_channels': in_channels, 
                    'out_dims': out_dims, 
                    'seq_length': cfg_data['seq_length']}

cfg_model_pool = []

### 1D Tiny CNN

In [8]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-Tiny-CNN'
# cfg_model['generator'] = TinyCNN1D
# cfg_model['fc_stages'] = 1
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 64
# cfg_model['LR'] = None
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

### M5 model

In [9]:
# cfg_model = {}
# cfg_model.update(cfg_common_model)
# cfg_model['model'] = '1D-M5'
# cfg_model['generator'] = M5
# cfg_model['fc_stages'] = 1
# cfg_model['use_age'] = 'fc'
# cfg_model['final_pool'] = 'max'
# cfg_model['base_channels'] = 256
# cfg_model['LR'] = None
# cfg_model['activation'] = 'relu'

# pprint.pprint('Model config:')
# pprint.pprint(cfg_model)
# print('\n' + '-' * 100 + '\n')
    
# model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
# print(model)
# print('\n' + '-' * 100 + '\n')

# del model
# cfg_model_pool.append(cfg_model)

### 1D ResNet variants

In [10]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-18'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BasicBlock1D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BasicBlock1D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'in_channels': 20,
 'model': '1D-ResNet-18',
 'out_dims': 3,
 'seq_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BasicBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(9,), stride=(3,), padding=(4,), bias=False)
      (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(1,), padding=(4,), bias=False)
     

In [11]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-50'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'in_channels': 20,
 'model': '1D-ResNet-50',
 'out_dims': 3,
 'seq_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(3,), padding=(4,), bias=False)
      (no

In [12]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-101'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'in_channels': 20,
 'model': '1D-ResNet-101',
 'out_dims': 3,
 'seq_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
      (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(64, 64, kernel_size=(9,), stride=(3,), padding=(4,), bias=False)
      (

### 1D ResNeXt variants

In [13]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-50'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 4
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '1D-ResNeXt-50',
 'out_dims': 3,
 'seq_length': 2000,
 'use_age': 'fc',
 'width_per_group': 4}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 128, kernel_size=(1,), stride=(1,), bias=False)
      (norm1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(128, 128, kernel_size=(9,), strid

In [14]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-101'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 8
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '1D-ResNeXt-101',
 'out_dims': 3,
 'seq_length': 2000,
 'use_age': 'fc',
 'width_per_group': 8}

----------------------------------------------------------------------------------------------------

ResNet1D(
  (conv0): Conv1d(20, 64, kernel_size=(41,), stride=(10,), padding=(20,), bias=False)
  (norm0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BottleneckBlock1D(
      (conv1): Conv1d(64, 256, kernel_size=(1,), stride=(1,), bias=False)
      (norm1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv1d(256, 256, kernel_size=(9,), str

### 2D ResNet variants

In [15]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-18' # resnet-18 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = BasicBlock2D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.BasicBlock2D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'in_channels': 20,
 'model': '2D-ResNet-18',
 'out_dims': 3,
 'seq_length': 2000,
 'stft_params': {'complex_mode': 'as_real', 'n_fft': 200},
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv0): Conv2d(20, 64, kernel_size=(7, 7), stride=(5, 5), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): BasicBlock2D(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1), bias=False)
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv2d(64

In [16]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-50' # resnet-50 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
    'hop_length': 100,
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.Bottleneck2D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'in_channels': 20,
 'model': '2D-ResNet-50',
 'out_dims': 3,
 'seq_length': 2000,
 'stft_params': {'complex_mode': 'as_real', 'hop_length': 100, 'n_fft': 200},
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv0): Conv2d(20, 64, kernel_size=(7, 7), stride=(5, 5), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): Bottleneck2D(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()
      (conv2): Conv2d

### 2D ResNeXt variants

In [17]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNeXt-50' # resnet-101 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 4
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.Bottleneck2D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '2D-ResNeXt-50',
 'out_dims': 3,
 'seq_length': 2000,
 'stft_params': {'complex_mode': 'as_real', 'n_fft': 200},
 'use_age': 'fc',
 'width_per_group': 4}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv0): Conv2d(20, 64, kernel_size=(7, 7), stride=(5, 5), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): Bottleneck2D(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU()

In [18]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNeXt-101' # resnet-101 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 8
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'
cfg_model['stft_params'] = {
    'n_fft': 200, 
    'complex_mode': 'as_real',  # 'as_real', 'power', 'remove'
}

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 64,
 'block': <class 'models.resnet_2d.Bottleneck2D'>,
 'conv_layers': [3, 4, 23, 3],
 'fc_stages': 3,
 'final_pool': 'max',
 'generator': <class 'models.resnet_2d.ResNet2D'>,
 'groups': 32,
 'in_channels': 20,
 'model': '2D-ResNeXt-101',
 'out_dims': 3,
 'seq_length': 2000,
 'stft_params': {'complex_mode': 'as_real', 'n_fft': 200},
 'use_age': 'fc',
 'width_per_group': 8}

----------------------------------------------------------------------------------------------------

ResNet2D(
  (conv0): Conv2d(20, 64, kernel_size=(7, 7), stride=(5, 5), padding=(3, 3), bias=False)
  (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv_stage1): Sequential(
    (0): Bottleneck2D(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU

#### CNN-Transformer

In [19]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-CNN-Transformer'
cfg_model['generator'] = CNNTransformer
cfg_model['fc_stages'] = 2
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 256  #
cfg_model['n_encoders'] = 8  #
cfg_model['n_heads'] = 8     #
cfg_model['dropout'] = 0.2   #
cfg_model['LR'] = None
cfg_model['activation'] = 'relu'

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

del model
cfg_model_pool.append(cfg_model)

'Model config:'
{'LR': None,
 'activation': 'relu',
 'base_channels': 256,
 'dropout': 0.2,
 'fc_stages': 2,
 'final_pool': 'max',
 'generator': <class 'models.cnn_transformer.CNNTransformer'>,
 'in_channels': 20,
 'model': '1D-CNN-Transformer',
 'n_encoders': 8,
 'n_heads': 8,
 'out_dims': 3,
 'seq_length': 2000,
 'use_age': 'fc'}

----------------------------------------------------------------------------------------------------

CNNTransformer(
  (conv0): Conv1d(20, 256, kernel_size=(21,), stride=(6,), padding=(10,), bias=False)
  (norm0): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act0): ReLU()
  (conv1): Conv1d(256, 512, kernel_size=(9,), stride=(2,), padding=(4,), bias=False)
  (norm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU()
  (conv2): Conv1d(512, 1024, kernel_size=(9,), stride=(2,), padding=(4,), bias=False)
  (norm2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track

#### Summarize the loaded models

In [20]:
for cfg_model in cfg_model_pool:
    pprint.pp(cfg_model, width=150)
    print('\n' + '-' * 100 + '\n')

{'in_channels': 20,
 'out_dims': 3,
 'seq_length': 2000,
 'model': '1D-ResNet-18',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'block': <class 'models.resnet_1d.BasicBlock1D'>,
 'conv_layers': [2, 2, 2, 2],
 'fc_stages': 3,
 'use_age': 'fc',
 'final_pool': 'max',
 'base_channels': 64,
 'LR': None,
 'activation': 'relu'}

----------------------------------------------------------------------------------------------------

{'in_channels': 20,
 'out_dims': 3,
 'seq_length': 2000,
 'model': '1D-ResNet-50',
 'generator': <class 'models.resnet_1d.ResNet1D'>,
 'block': <class 'models.resnet_1d.BottleneckBlock1D'>,
 'conv_layers': [3, 4, 6, 3],
 'fc_stages': 3,
 'use_age': 'fc',
 'final_pool': 'max',
 'base_channels': 64,
 'LR': None,
 'activation': 'relu'}

----------------------------------------------------------------------------------------------------

{'in_channels': 20,
 'out_dims': 3,
 'seq_length': 2000,
 'model': '1D-ResNet-101',
 'generator': <class 'models.resnet_1d.ResNet

-----

## Default Configurations for Training

In [21]:
# training configurations
cfg_train = {}
cfg_train['iterations'] = 200000  # (300000 * 32) // cfg_data['minibatch']
cfg_train['warmup_steps'] = max(cfg_train['iterations'] * 0.05, 3000)
cfg_train['num_history'] = 500
cfg_train['lr_scheduler_type'] = 'constant_with_decay'
cfg_train['weight_decay'] = 1e-2
cfg_train['mixup'] = 0.0  # 0.0 for no usage
cfg_train['criterion'] = 'cross-entropy' # 'cross-entropy', 'multi-bce'

cfg_train['device'] = device
cfg_train['save_model'] = True
cfg_train['draw_result'] = True
cfg_train['watch_model'] = False

-----

## Train

In [22]:
sweep_data = {}
sweep_data['seq_length'] = {
    'values': [
        200 *  5, #  5 sec
        200 * 10, # 10 sec
        200 * 15, # 15 sec
        200 * 20, # 20 sec
        200 * 30, # 30 sec
              ],
}

sweep_data['EKG'] = {
    'values': ['O', 'X'],
}

sweep_data['photic'] = {
    'values': ['O', 'X'],
}

sweep_data['awgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

sweep_data['mgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.1,
}

sweep_data['awgn_age'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

In [23]:
sweep_model = {}
sweep_model['model_index'] = { 
    'values': [i for i in range(len(cfg_model_pool))] 
}

sweep_model['fc_stages'] = { 
    'distribution': 'int_uniform',
    'min': 1,
    'max': 4,
}

sweep_model['use_age'] = { 
    'values': ['fc', 'conv', 'no']
}

sweep_model['base_channels'] = { 
    'values': [64, 128]
}

sweep_model['dropout'] = {
    'values': [0, 0.1, 0.2, 0.3, 0.4, 0.5]
}

# sweep_model['LR'] = {
#     'distribution': 'log_uniform',
#     'min': math.log(5e-5),
#     'max': math.log(2e-3)
# }

sweep_model['activation'] = {
    'values': ['relu', 'gelu', 'mish']
}

In [24]:
sweep_train = {}

sweep_train['lr_scheduler_type'] = {
    'values': [
        'constant_with_decay',
        'constant_with_twice_decay',
        'transformer_style',
        'cosine_decay_with_warmup_half',
        'cosine_decay_with_warmup_one_and_half',
#        'cosine_decay_with_warmup_two_and_half',
        'linear_decay_with_warmup',
    ]

}

sweep_train['search_multiplier'] = {
    'values': [0.8, 1.0, 1.2, 2.0]
}

sweep_train['weight_decay'] = {
    'distribution' : 'log_uniform_values',
    'min': 1e-5,
    'max': 1e-1
}

sweep_train['mixup'] = {
    'values': [0, 0.1, 0.2, 0.3]
}

sweep_train['criterion'] = {
    'values': ['cross-entropy', 'multi-bce']
}

In [25]:
sweep_config = {
    "entity": "ipis-mjkim",
    "name" : "my-sweep",
    "method": "random",
    "parameters": 
    {
        **sweep_data,
        **sweep_model,
        **sweep_train,
    }
}

sweep_id = wandb.sweep(sweep_config, project=f"caueeg-{cfg_data['task']}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: wq4qgtfq
Sweep URL: https://wandb.ai/ipis-mjkim/caueeg-task1/sweeps/wq4qgtfq


In [26]:
def train():
    # initialize the wandb log
    wandb_run = wandb.init()
    wandb.run.name = wandb.run.id

    with wandb_run:
        # collect some garbages
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    
        # load default config
        config = {}
        cfg_model = cfg_model_pool[wandb.config.model_index]
        for k, v in {**cfg_data,  **cfg_model, **cfg_train}.items():
            if k not in wandb.config:
                config[k] = v
                
        # load the selected configurations from wandb sweep with preventing callables from type-conversion to str
        for k, v in wandb.config.items():
            if k not in config:
                config[k] = v

        # build dataset
        train_loader, val_loader, test_loader, test_loader_longer = build_dataset_for_train(config)
        
        # train the model
        model = train_with_wandb(config, train_loader, val_loader, test_loader, test_loader_longer, 
                                 config['preprocess_train'], config['preprocess_test'])
        
        # release memory
        del model
        del train_loader, val_loader
        del test_loader, test_loader_longer
        del config
        
    time.sleep(60)

In [ ]:
wandb.agent(sweep_id, function=train, count=100)

wandb: Agent Starting Run: ih3x9v4t with config:
wandb: 	EKG: O
wandb: 	activation: relu
wandb: 	awgn: 0.0008718713775749731
wandb: 	awgn_age: 0.07558424497197733
wandb: 	base_channels: 128
wandb: 	criterion: cross-entropy
wandb: 	dropout: 0.2
wandb: 	fc_stages: 1
wandb: 	lr_scheduler_type: linear_decay_with_warmup
wandb: 	mgn: 0.01176988468371898
wandb: 	mixup: 0
wandb: 	model_index: 7
wandb: 	photic: O
wandb: 	search_multiplier: 2
wandb: 	seq_length: 1000
wandb: 	use_age: fc
wandb: 	weight_decay: 0.015993380634519824
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ipis-mjkim (use `wandb login --relogin` to force relogin)


************************************************************************************************************************
******************************                 2D-ResNeXt-50 train starts                 ******************************
************************************************************************************************************************
